# text_translation

https://www.kaggle.com/competitions/predict-student-performance-from-game-play/discussion/396068  
モードごとのtextの違いを名寄せする

In [21]:
import os
import sys
import traceback
import gc
import random
import pickle
import pathlib
import subprocess
from dataclasses import dataclass

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [22]:
@dataclass
class Cfg:
    input_dir = "/mnt/predict-student-performance-from-game-play/input/"
    output_dir = "/mnt/predict-student-performance-from-game-play/output/"
    prep_dir = "/mnt/predict-student-performance-from-game-play/prep/"
cfg = Cfg()

In [23]:
df

,normal,dry,nohumor,nosnark
0,Gah. I can't believe this.,Gah. I can't believe this.,I can't believe this.,Gah. I can't believe this.
1,Then who took Teddy???,Then who took Teddy?,Then who took Teddy?,Then who took Teddy???
2,"Um, are you okay?","Um, are you okay?","Um, are you okay?",Are you okay?
3,You look like you're on a mission.,How can I help you?,How can I help you?,You look like you're on a mission.
4,Not really...,"Sure, Gramps!",Not really...,YES!!!
...,...,...,...,...
71,Whatever.,I can't believe this.,Whatever.,Whatever.
72,Grab your notebook and come upstairs!,"Sure thing, Jo. Grab your notebook and come up...",Grab your notebook and come upstairs!,"Sure thing, Jo. Grab your notebook and come up..."
73,"Alright, everybody hold up-",Wait a minute.,Wait a minute.,"Alright, everybody hold up-"
74,Gadzooks! Poor critter.,Poor badger.,Poor badger.,Gadzooks! Poor critter.


In [24]:
def get_text_translation_map():
    df = pd.read_csv(cfg.prep_dir + "game_type_text_translation.csv")
    text_translation_map = {}
    text_translation_map.update({a:b for a, b in zip(df["dry"].values, df["normal"].values)})
    text_translation_map.update({a:b for a, b in zip(df["nohumor"].values, df["normal"].values)})
    text_translation_map.update({a:b for a, b in zip(df["nosnark"].values, df["normal"].values)})
    return text_translation_map

In [25]:
text_translation_map = get_text_translation_map()

In [26]:
train = pd.read_csv(cfg.input_dir + "train.csv")

In [27]:
train["text"].nunique()

597

In [28]:
train["text"] = train["text"].replace(text_translation_map)

In [29]:
train["text"].nunique()

538

In [33]:
train["text"].unique().tolist()

['undefined',
 'Whatcha doing over there, Jo?',
 'Just talking to Teddy.',
 'I gotta run to my meeting!',
 'Meetings are BORING!',
 'Grab your notebook and come upstairs!',
 'Hang tight, Teddy.',
 "I'll hurry back and then we can go exploring!",
 'Now where did I put my notebook?',
 '\\u00f0\\u0178\\u02dc\\u00b4',
 nan,
 'I love these photos of me and Teddy!',
 'Found it!',
 'Gramps is in trouble for losing papers?',
 "This can't be right!",
 'Gramps is a great historian!',
 "Hmm. Button's still not working.",
 "Let's get started. The Wisconsin Wonders exhibit opens tomorrow!",
 'Who wants to investigate the shirt artifact?',
 "Not Leopold here. He's been losing papers lately.",
 'Hey!',
 "It's true, they do keep going missing lately.",
 'See?',
 'Besides, I already figured out the shirt.',
 "It's a women's basketball jersey!",
 'That settles it.',
 'Wells, finish up your report.',
 "Leopold, why don't you help me set up in the Capitol?",
 'We need to talk about that missing paperwork.